In [1]:
import web3
from pycoingecko import CoinGeckoAPI
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import requests
from addict import Addict as ad
import time
import copy
import tqdm
import collections

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

### Coingecko API Prices

In [2]:
ids = [
    'binancecoin',
    'wbnb',
    'synclub-staked-bnb',
    'tether',
    'usd-coin',
    'helio-protocol-hay',
    'frax',
    'coin98-dollar',
    'usdtplus',
    'usd',
    'wombat-exchange',
    'wombex'
]

cg = CoinGeckoAPI()
def get_crypto_info(crypto_id):
    crypto_data = cg.get_coin_by_id(crypto_id)
    price_data = crypto_data["market_data"]["current_price"]
    usd_price = price_data.get("usd")
    symbol = crypto_data["symbol"].upper()
    return symbol, usd_price

prices = pd.DataFrame(columns=['symbol','underlyingPrice'])

for idx, crypto_id in enumerate(ids):
    symbol, price = get_crypto_info(crypto_id)
    prices.loc[idx] = [symbol, price]

prices

,symbol,underlyingPrice
0,BNB,214.710000
1,WBNB,214.660000
2,SNBNB,214.380000
3,USDT,0.999530
4,USDC,0.999625
5,HAY,0.999384
6,FRAX,0.998536
7,CUSD,1.006000
8,USDT+,0.997374
9,USD+,0.999054


In [3]:
wom_price = prices.loc[prices['symbol'] == 'WOM', 'underlyingPrice'].item()
wmx_price = prices.loc[prices['symbol'] == 'WMX', 'underlyingPrice'].item()

new_ids = [16,17,26,27,28,34,5,36,37,44,45]

### Web3

In [4]:
rpc = 'https://bsc-dataseed1.binance.org/' #'https://bscrpc.com'
endpoint = 'https://api.bscscan.com/api?module=contract&action=getabi&address='
abis = ad({})
contracts = ad({})

w3cs = web3.Web3(web3.HTTPProvider(rpc))
w3cs.middleware_onion.inject(web3.middleware.geth_poa_middleware, layer = 0)
print('connection status:', w3cs.is_connected())

connection status: True


In [5]:
addresses = ad({
        'booster':'0xa4a1533f5f939d6718b0d5ce2850f2ff55206967',
        'wombatRouter': '0x19609B03C976CCA288fbDae5c21d4290e9a4aDD7',
        'wombatMaster': '0x489833311676B566f888119c29bd997Dc6C95830',
        'wmx': '0xa75d9ca2a0a1D547409D82e1B06618EC284A2CeD',
        'pancake': '0x10ED43C718714eb63d5aA57B78B54704E256024E',
        'gaugeVoting': '0x6d1fce96e26d7e48e8ecc88a7d9d8241c00e9af8'
    })

implementations = ad({
        'booster':'0xa4a1533f5f939d6718b0d5ce2850f2ff55206967',
        'wombatRouter': '0x19609B03C976CCA288fbDae5c21d4290e9a4aDD7',
        'wombatMaster': '0x55e4aC63b9275BA52223980e603F9eeB3157FE7B',
        'wmx': '0xa75d9ca2a0a1D547409D82e1B06618EC284A2CeD',
        'pancake': '0x10ED43C718714eb63d5aA57B78B54704E256024E',
        'gaugeVoting': '0x6d1fce96e26d7e48e8ecc88a7d9d8241c00e9af8',
        'wmxPool': '0xc9DcF462B666aB2196E044AFcF7814F60cd2E09d',
        'wmxBribePool': '0x1623955a87DC65B19482864d7a1F7213F0e3e04A',
        'womPool': '0xCD32277631da27b4cC4a229B2dcf1E8EcE3CDA5C',
        'womLP': '0xA649Be04619a8F3B3475498E1ac15C90C9661C1A'
    })

for imp in implementations:
    abi = json.loads(requests.get(endpoint+implementations[imp]).json()['result'])
    abis[imp] = abi
    time.sleep(5.1)

for addr in addresses:
    contracts[addr] = w3cs.eth.contract(abi = abis[addr], address = w3cs.to_checksum_address(addresses[addr]))

In [6]:
def getPoolInfo(pools, verbose = True):
    info = []
    denom = contracts.booster.functions.DENOMINATOR().call()
    wmxMnt = contracts.wmx.functions.getFactAmounMint(int(1e18)).call()/1e18
    
    for i in tqdm.tqdm(pools):
        customRatio = contracts.booster.functions.customMintRatio(i).call()
        mintRatio = customRatio if customRatio > 0 else contracts.booster.functions.mintRatio().call()
        poolInfo = contracts.booster.functions.poolInfo(i).call()
        wmx_pid = i
        lpToken = poolInfo[0]
        wom_pid = contracts.wombatMaster.functions.getAssetPid(lpToken).call()
        wmxLp = poolInfo[1]
        rewardPool = poolInfo[-2]
        womLpC = w3cs.eth.contract(abi = abis.womLP, address = lpToken)
        underlyingToken = womLpC.functions.underlyingToken().call()
        symbol = womLpC.functions.symbol().call()
        lpC = w3cs.eth.contract(abi = abis.wmxPool, address = rewardPool)
        wmxLP_amount = (lpC.functions.totalSupply().call())/1e18
        rewardTokens = lpC.functions.rewardTokensList().call()
        other_per_year = 0
        other_symbol = '-'
        for rewardToken in rewardTokens:
            rtc = w3cs.eth.contract(address = rewardToken, abi = abis.womLP)
            rtdec = rtc.functions.decimals().call()
            rtsym = rtc.functions.symbol().call()
            rtInfo = lpC.functions.tokenRewards(rewardToken).call()
            #rtInfo[-1]=True == rewards in this token are paused
            #rtInfo[1]<w3cs.eth.get_block('latest').timestamp == period finished
            rewardRate = rtInfo[2]/(10**rtdec)*(not rtInfo[-1])*(rtInfo[1]>=w3cs.eth.get_block('latest').timestamp)
            if rtsym.lower() == 'wom':
                wom_per_year = rewardRate*3600*24*365
                wmx_per_year = wom_per_year*wmxMnt*mintRatio/denom
            else:
                other_per_year = rewardRate*3600*24*365
                other_symbol = rtsym
        if poolInfo[-1] == False: #pool is not shutdown
            info.append({'wom_pid': wom_pid, 'lpToken': lpToken, 'lpSymbol': symbol, 'wmxLp': wmxLp, 'rewardPool': rewardPool, 'wmx_pid': wmx_pid, 'wmxLP_amount': wmxLP_amount, 'underlyingToken': underlyingToken, 'womPerYear': wom_per_year, 'wmxPerYear': wmx_per_year, 'otherPerYear': other_per_year, 'otherSymbol': other_symbol})
    return info

In [7]:
bsc_info = getPoolInfo(new_ids)
bsc_info = pd.DataFrame(bsc_info)
bsc_info['rewardPool'] = bsc_info['rewardPool'].str.lower()
bsc_info['underlyingSymbol'] = bsc_info['lpSymbol'].str[3:].str.upper()

tab = bsc_info.merge(prices, how='left', left_on = 'underlyingSymbol', right_on = 'symbol')
tab['TVL'] = tab['wmxLP_amount']*tab['underlyingPrice']
tab['WOM_APR'] = round(100*tab['womPerYear']*wom_price/tab['TVL'], 2)
tab['WMX_APR'] = round(100*tab['wmxPerYear']*wmx_price/tab['TVL'], 2)
tab['other_APR'] = round(100*tab['otherPerYear']*tab['underlyingPrice']/tab['TVL'], 2)
tab['Total_APR'] = tab['WOM_APR'] + tab['WMX_APR'] + tab['other_APR']
tab.drop(['symbol', 'womPerYear', 'wmxPerYear', 'otherPerYear', 'otherSymbol'], axis=1, inplace=True)
tab

100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [00:30<00:00,  2.74s/it]


,wom_pid,lpToken,lpSymbol,wmxLp,rewardPool,wmx_pid,wmxLP_amount,underlyingToken,underlyingSymbol,underlyingPrice,TVL,WOM_APR,WMX_APR,other_APR,Total_APR
0,21,0x3ac762C607ed6Dba156cBcF11efF96340e86b490,LP-CUSD,0x65bc698691Ed11c1d57e5B797A6Bc32F100590D3,0xa79a406da57394a38a68d2d6bf3e34a801f39219,16,21283.914460,0xFa4BA88Cf97e282c505BEa095297786c16070129,CUSD,1.006000,21411.617947,0.00,0.00,10.94,10.94
1,22,0xa6eF6C45EbFDBc13f6D032fbDFeC9b389C1603E5,LP-HAY,0x5447a06a20A2854b59804606FFA1B8e35CF898C2,0x25622a2789e647bc7f675952f048fdd3e3e2ddfe,17,41204.425806,0x0782b6d8c4551B9760e74c0545a9bCD90bdc41E5,HAY,0.999384,41179.043880,0.00,0.00,0.00,0.00
2,34,0x88bEb144352BD3109c79076202Fac2bcEAb87117,LP-USD+,0xf5Fdb16e028aAA2651f1e429a6eb49fC00163be6,0xed2510a5a107ab273a1bf69b14651d7592d55476,26,11.593800,0xe80772Eaf6e2E18B651F160Bc9158b2A5caFCA65,USD+,0.999054,11.582832,0.00,0.00,0.00,0.00
3,35,0xbd459E33307A4ae92fFFCb45C6893084CFC273B1,LP-USDT+,0x0EC9DB121f8Fb5de286c9A0d6Bcc6354E78d4C9C,0xf73dc098aeb7b61155770b8efba4c5291cd08bd6,27,17.613985,0x5335E87930b410b8C5BB4D43c3360ACa15ec0C8C,USDT+,0.997374,17.567730,0.00,0.00,0.00,0.00
4,36,0x8Df8b50B73849f0433EE3314BD956e624e67b3ce,LP-USDC,0x23Ef66A02f9A7d1A03964aB79D31f2d47c09544B,0xe8cd770a1210ed3417fafe1752551e61c6fc9ae5,28,9960.102348,0x8AC76a51cc950d9822D68b83fE1Ad97B32Cd580d,USDC,0.999625,9956.367310,2.15,1.59,0.00,3.74
5,49,0x6b60066966080deaB5090d6026CB134591a1cC95,LP-FRAX,0x67b26A46FFFbCE9987E6d1f1eB47e6b00736E8Fb,0x34431ae4a37f177010978ef2af9fb31ba5a066f5,34,13293.063614,0x90C97F71E18723b0Cf0dfa30ee176Ab653E89F40,FRAX,0.998536,13273.602569,0.00,0.00,0.00,0.00
6,5,0x1fa71DF4b344ffa5755726Ea7a9a56fbbEe0D38b,LP-HAY,0xD4A614Fd9f9B16c0e69BA08CdD4152EDb0E53372,0xc9dcf462b666ab2196e044afcf7814f60cd2e09d,5,174849.315688,0x0782b6d8c4551B9760e74c0545a9bCD90bdc41E5,HAY,0.999384,174741.608509,11.34,8.37,0.00,19.71
7,42,0x184EB7DDE8c00fad900120235b534AABc1d96F75,LP-USDT,0xd42F96eA0C292E75485103eD2d9b1AE51428bFC4,0x11c8c9e576287d3e12f3b53fe58840ef5b34f508,36,248136.403869,0x55d398326f99059fF775485246999027B3197955,USDT,0.999530,248019.779760,9.48,6.99,0.00,16.47
8,43,0x681124f5BDD9aCDA19d1721063254189903CD685,LP-USDC,0x19110dB9b19AA2934abF7a66886535F15b482d28,0xcc3796a74a08bf30be33ac8d6d433f42231b5f15,37,238255.960684,0x8AC76a51cc950d9822D68b83fE1Ad97B32Cd580d,USDC,0.999625,238166.614698,12.71,9.38,0.00,22.09
9,50,0xaA0811AfF60Fbe2d7D7D0A18F26e584b8C148Ee8,LP-SnBNB,0x743E1c450c2b037B08Fec761A14309dEF221cCea,0x834a0d345263a0e0652d3ab1b4f003b7e285c72b,44,165.857523,0xB0b84D294e0C75A6abe60171b70edEb2EFd14A1B,SNBNB,214.380000,35556.535812,21.64,15.97,0.00,37.61


In [8]:
tab1 = tab.to_json(orient='records')
tab2 = json.loads(tab1)
tab3 = json.dumps(tab2, indent=2)
print(tab3)


with open('full_info.json', 'w', encoding='utf-8') as f:
    json.dump(tab3, f, ensure_ascii=False, indent=4)

[
  {
    "wom_pid": 21,
    "lpToken": "0x3ac762C607ed6Dba156cBcF11efF96340e86b490",
    "lpSymbol": "LP-CUSD",
    "wmxLp": "0x65bc698691Ed11c1d57e5B797A6Bc32F100590D3",
    "rewardPool": "0xa79a406da57394a38a68d2d6bf3e34a801f39219",
    "wmx_pid": 16,
    "wmxLP_amount": 21283.914460196,
    "underlyingToken": "0xFa4BA88Cf97e282c505BEa095297786c16070129",
    "underlyingSymbol": "CUSD",
    "underlyingPrice": 1.006,
    "TVL": 21411.6179469572,
    "WOM_APR": 0.0,
    "WMX_APR": 0.0,
    "other_APR": 10.94,
    "Total_APR": 10.94
  },
  {
    "wom_pid": 22,
    "lpToken": "0xa6eF6C45EbFDBc13f6D032fbDFeC9b389C1603E5",
    "lpSymbol": "LP-HAY",
    "wmxLp": "0x5447a06a20A2854b59804606FFA1B8e35CF898C2",
    "rewardPool": "0x25622a2789e647bc7f675952f048fdd3e3e2ddfe",
    "wmx_pid": 17,
    "wmxLP_amount": 41204.4258062495,
    "underlyingToken": "0x0782b6d8c4551B9760e74c0545a9bCD90bdc41E5",
    "underlyingSymbol": "HAY",
    "underlyingPrice": 0.999384,
    "TVL": 41179.0438799528,
    